# Patient Administrative Outcomes Predictive Model

# 00. Setup Development Environment 

- Install Poetry to simplify dependency management. 
    - `pip install poetry` 
- Setup Poetery Project 
    - `poetry init` 
- Create a virtual envrionment 
    - `python -m venv .venv` 
- Add dependencies 
    - `poetry add pyspark ipykernel pandas matplotlib ploty seaborn`


In [130]:
import os
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

import warnings
warnings.filterwarnings('ignore')

# 01. Data Integration

<!-- Data integration with Fast Healthcare Interoperability Resources (FHIR) resources involves unifying healthcare data from multiple systems to enable seamless data exchange and interoperability. This approach enhances data consistency, accessibility, and streamlines access to critical healthcare information across applications and organizations.

**Data Source: MIMIC-IV on FHIR**

The project uses MIMIC-IV on FHIR as its primary data source. This dataset provides a rich, de-identified healthcare database mapped to the FHIR (Fast Healthcare Interoperability Resources) standard to support interoperability and advanced analytics in healthcare research.

MIMIC-IV on FHIR is a project by PhysioNet designed to translate the structure and content of MIMIC-IV into the FHIR standard. MIMIC-IV is a relational database containing real hospital stay data for patients admitted to a tertiary academic medical center in Boston, MA, USA. It includes comprehensive clinical and administrative information recorded between 2008 and 2019.

By leveraging MIMIC-IV on FHIR, this project aims to enhance data interoperability and enable the development of predictive models and analytical insights aligned with modern healthcare standards and regulatory compliance.

**Fast Healthcare Interoperability Resources (FHIR)**

FHIR is a specification for exchanging health care data across different systems and platforms. The specification builds on and adapts modern, widely used RESTful practices to enable the provision of integrated healthcare across a wide range of teams and organizations. FHIR is based on "Resources" which are the common building blocks for all exchanges.

Key FHIR Resources for Integration:

- Patient: Stores demographic information such as name, age, and contact details.
- Encounter: Captures details of patient visits and interactions with healthcare providers.
- Condition: Represents diagnoses and health conditions affecting the patient.
- Procedure: Records medical procedures and interventions performed.
- Observation: Contains lab results, vital signs, and other clinical observations.

**Medallion Architecture**

The project follows a Medallion Architecture framework, which organizes data into multiple layers to ensure structured data refinement, improved data quality, and optimized analytics. This approach enhances data governance and enables efficient processing for downstream use cases.

The architecture comprises three key layers:
1. Bronze Layer (Raw Data): The Bronze layer stores raw, ingested data from various source systems without transformations. It retains all original fields and formats, providing a comprehensive source of truth for historical and traceability purposes.

2. Silver Layer (Filtered, Cleaned, Augmented): The Silver layer refines and transforms the raw data, selecting only the necessary fields from relevant FHIR resources (e.g., Patient, Encounter, Condition, Procedure). This layer provides clean, structured, and domain-specific datasets for targeted analysis and processing.

3. Gold Layer (Business-Level Aggregrates): The Gold layer aggregates data into a unified, optimized format for advanced analytics and reporting. In this project, it focuses on a Consolidated Encounter Table, combining encounter data with relevant patient demographics, conditions, and other clinical information to support predictive modeling and operational insights.

This layered architecture promotes scalability, data quality, and efficient data retrieval, aligning with best practices for healthcare data management and analytics.

> **References**
> - [MIMIC Implementatoin Guide | KinD Lab](https://mimic.mit.edu/fhir/)
> - [FHIR Documentation | HL7FHIR](https://www.hl7.org/fhir/)
> - [Medallion Architecture | Databricks](https://www.databricks.com/glossary/medallion-architecture) -->

In [2]:
spark = SparkSession.builder \
    .appName("FhirDataApplication") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.2") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.local.type", "hadoop") \
    .config("spark.sql.catalog.local.warehouse", "dataset/iceberg") \
    .getOrCreate()

## Bronze Layer

<!-- The Bronze Layer serves as the foundation of the data architecture, capturing raw, untransformed data from various source systems. It retains all original fields and formats, providing a comprehensive source of truth for historical and traceability purposes. This layer plays a crucial role in maintaining data integrity, enabling auditability, and supporting complex analytical workflows in later stages.

In the project, `bronze.resource` serves as a storage for raw data that has been ingested from various source files. -->

In [ ]:
with open("sql_queries/bronze.resource.ddl.sql") as f:
    spark.sql(f.read())

In [ ]:
RESOURCE_FILES = ["MimicPatient.ndjson.gz", "MimicCondition.ndjson.gz", "MimicConditionED.ndjson.gz", "MimicEncounter.ndjson.gz", "MimicEncounterED.ndjson.gz", "MimicEncounterICU.ndjson.gz"] 
for resource_file in RESOURCE_FILES: 
    resource_df = spark.read.text(f"dataset/{resource_file}") 
    resource_df = resource_df.withColumn("sourceFile", F.lit(resource_file)) 
    resource_df.write.format("iceberg").mode("append").save("local.bronze.resource")

In [ ]:
print("Data Volume in Each File")
data_volume_df = spark.sql("select sourceFile, count(*) cnt from  local.bronze.resource group by 1").toPandas()

In [ ]:
plt.figure(figsize=(20, 8))
bars = plt.bar(data_volume_df['sourceFile'], data_volume_df['cnt'])

plt.xlabel('Categories')
plt.ylabel('Values')
plt.title('Bar Chart with Full Number Labels')

for bar in bars:
    plt.text(
        bar.get_x() + bar.get_width() / 2,  # X position
        bar.get_height(),                   # Y position
        f'{bar.get_height():,}',            # Format with commas
        ha='center', va='bottom', fontsize=10  # Center alignment, font size
    )

## Silver Layer

<!-- The Silver Table represents the refined data layer in the Medallion Architecture, where raw data from the Bronze Layer is transformed and cleansed. It focuses on structuring the data and selecting only the necessary fields from relevant sources, ensuring that it is ready for more detailed analysis and reporting.

The Silver Tables in use include Patient, Encounter, and Condition. The Patient table stores essential demographic information, while the Encounter table tracks patient visits and interactions with healthcare providers. The Condition table holds information about patient diagnoses and health conditions.

The resources are stored while adhering to Safe Harbor provisions to ensure patient privacy and compliance with data protection regulations.

**Safe Harbor Provisions**

Safe Harbor provisions are a set of guidelines that allow organizations to de-identify personal data in a way that ensures compliance with privacy regulations, particularly in the context of healthcare and data protection laws like HIPAA. Under these provisions, data can be stripped of specific identifiers, such as names, addresses, and Social Security numbers, making it no longer considered protected health information (PHI). Safe Harbor provisions offer a balance between privacy protection and the need for data availability, enabling the use of anonymized data while minimizing the risk of re-identification.


> **References**
> - [Guidance Regarding Methods for De-identification of PHI](https://www.hhs.gov/hipaa/for-professionals/special-topics/de-identification/index.html) -->

In [ ]:
# Load Bronze Table
raw_df = spark.read.format("iceberg").load("local.bronze.resource")

### Patient Mart

<!-- The Patient Mart stores essential demographic information for each patient, providing a foundational set of details necessary for understanding patient profiles and is used in various healthcare processes, from clinical interactions to research and analysis.

| Column Name   | Description                                   |
|---------------|-----------------------------------------------|
| patientID     | A unique identifier assigned to each patient. |
| gender        | The patient's gender (e.g., male or female).  |
| birthDate     | The patient's date of birth.                  |
| maritalStatus | The marital status of the patient.            | -->

In [ ]:
with open("sql_queries/silver.patient.ddl.sql") as f:
    spark.sql(f.read())

In [ ]:
patient_df = raw_df.filter(raw_df.sourceFile.isin("MimicPatient.ndjson.gz"))
with open("schema/Patient.json") as f:
    schema_read = json.loads(f.read())
schema = T.StructType.fromJson(schema_read)
patient_df = patient_df.withColumn("parsed_json", F.from_json(patient_df["value"], schema))
patient_df = patient_df.select("parsed_json.*", "sourceFile")
patient_df.createOrReplaceTempView("patient_df")

with open("sql_queries/silver.patient.load.sql") as f:
    fm_patient = spark.sql(f.read())

fm_patient.show(5, truncate=False)
fm_patient.write.format("iceberg").mode("overwrite").save("local.silver.patient")

### Encounter Mart

<!-- The Encounter resource represents an interaction between a patient and healthcare provider(s) for the purpose of delivering healthcare services or assessing the patient’s health status.

It captures the full span of a hospital stay, from admission to discharge, and includes important details such as the start and end time of the admission, the context surrounding the admission, and patient movements within the healthcare facility.

| Column            | Description                                                                                                                          |
|-------------------|--------------------------------------------------------------------------------------------------------------------------------------|
| encounterId       | A unique identifier for the encounter.                                                                                               |
| patientId         | A unique identifier for the patient involved in the encounter.                                                                       |
| ref_encounterId   | A reference to a related encounter, which may relate to both future and past encounters.                                             |
| periodStart       | The start timestamp marking the beginning of the encounter period.                                                                   |
| periodEnd         | The end timestamp marking the conclusion of the encounter period.                                                                    |
| duration          | The total duration of the encounter, typically in hours or days.                                                                     |
| status            | The current status of the encounter (e.g., planned, in-progress, finished).                                                          |
| encounterClass    | The classification of the encounter, such as inpatient or outpatient, helping to categorize the type of healthcare service provided. |
| codedType         | A code representing the specific type of encounter, aiding in standardized classification.                                           |
| displayType       | A human-readable display name for the type of encounter.                                                                             |
| systemType        | The system from which the encounter type code is derived.                                                                            |
| priority          | The urgency level of the encounter, which may include options like routine, urgent, or emergency.                                    |
| nextEncounterId   | A reference to next encounter                                                                                                        |
| readmissionStatus | Future readmission indicator                                                                                                         | -->

In [ ]:
with open("sql_queries/silver.encounter.ddl.sql") as f:
    spark.sql(f.read())

In [ ]:
encounter_df = raw_df.filter(raw_df.sourceFile.isin("MimicEncounter.ndjson.gz", "MimicEncounterED.ndjson.gz")) 
with open("schema/Encounter.json") as f: 
    schema_read = json.loads(f.read()) 
schema = T.StructType.fromJson(schema_read) 
encounter_df = encounter_df.withColumn("parsed_json", F.from_json(encounter_df["value"], schema)) 
encounter_df = encounter_df.select("parsed_json.*", "sourceFile") 
encounter_df.createOrReplaceTempView("encounter_df") 

with open("sql_queries/silver.encounter.load.sql") as f:
    fm_encounter = spark.sql(f.read())

fm_encounter.show(5, truncate=False) 
fm_encounter.write.format("iceberg").mode("overwrite").save("local.silver.encounter")

### Condition Mart

<!-- The Condition resource in FHIR is used to document a patient’s health state, including diagnoses, clinical problems, or other health concerns.

It captures conditions that are significant enough to warrant clinical focus, including diseases, health problems, or the state following a medical procedure.

This resource plays a critical role in capturing a patient’s health history and providing a detailed record of conditions for clinical decision-making, research, and care management.

| Column           | Description                                                                                 |
|------------------|---------------------------------------------------------------------------------------------|
| conditionId      | A unique identifier for the condition.                                                      |
| patientId        | A unique identifier for the patient associated with the condition.                          |
| encounterId      | The unique identifier for the encounter where the condition was diagnosed or recorded.      |
| categoryCode     | A code representing the category of the condition (e.g., diagnosis, problem).               |
| conditionCode    | The code that represents the specific condition, typically from a standardized code system. |
| conditionDisplay | A human-readable display name for the condition.                                            |
| conditionSystem  | The system from which the condition code is derived (e.g., ICD-10, SNOMED CT).              | -->

In [ ]:
with open("sql_queries/silver.condition.ddl.sql") as f:
    spark.sql(f.read())

In [ ]:
condition_df = raw_df.filter(raw_df.sourceFile.isin("MimicCondition.ndjson.gz", "MimicConditionED.ndjson.gz")) 
with open("schema/Condition.json") as f: 
    schema_read = json.loads(f.read()) 
schema = T.StructType.fromJson(schema_read) 
condition_df = condition_df.withColumn("parsed_json", F.from_json(condition_df["value"], schema)) 
condition_df = condition_df.select("parsed_json.*", "sourceFile") 
condition_df.createOrReplaceTempView("condition_df") 

with open("sql_queries/silver.condition.load.sql") as f:
    fm_condition = spark.sql(f.read())

fm_condition.show(5, truncate=False) 
fm_condition.write.format("iceberg").mode("overwrite").save("local.silver.condition")

# 02. Data Exploration

In [3]:
# !pip install plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Patient Dataset

In [ ]:
px.line(
    spark.sql("select date_format(birthDate, 'yyyy') birthYear, count(*) count from local.silver.patient group by 1 order by 1").toPandas(),
    x='birthYear',
    y='count',
    title='Patient BirthYear Distribution [De-Identified]'
).update_layout(
    xaxis_title='Birth Year',
    yaxis_title='Number of Patients',
    template='plotly_white'
).show()

In [ ]:
px.pie(
    spark.sql("select gender, count(*) count from local.silver.patient group by 1 order by 1").toPandas(),
    values='count', names='gender', title='Gender Distribution'
).show()

In [ ]:
px.bar( 
    spark.sql("select gender, maritalStatus, count(*) count from local.silver.patient group by 1,2").toPandas(), 
    x='maritalStatus', 
    y='count', 
    color='gender', 
    title='Count by Gender and Marital Status', 
    barmode='group' 
).show()

## Encounter Dataset

In [ ]:
px.line(
    spark.sql("""
    select date_format(periodStart, 'yyyy') encounterYear, count(*) count from local.silver.encounter group by 1 order by 1
    """).toPandas(),
    x='encounterYear',
    y='count',
    title='Encounter Period Year Distribution [De-Identified]'
).update_layout(
    xaxis_title='Year',
    yaxis_title='Number of Encounters',
    template='plotly_white'
).show()

In [ ]:
px.pie(
    spark.sql("select sourceFile, count(*) count from local.silver.encounter group by 1").toPandas(),
values='count', names='sourceFile', title='Source Distribution').show()

In [ ]:
px.pie(
    spark.sql("select encounterClass, count(*) count from local.silver.encounter group by 1").toPandas(),
values='count', names='encounterClass', title='Encounter Class Distribution').show()

In [ ]:
px.pie(
    spark.sql("select priority, count(*) count from local.silver.encounter group by 1").toPandas(),
values='count', names='priority', title='Encounter Class by Priority').show()

In [ ]:
px.pie(
    spark.sql("select readmissionStatus, count(*) count from local.silver.encounter group by 1").toPandas(),
values='count', names='readmissionStatus', title='Patient Readmission').show()

In [ ]:
px.line(
    spark.sql("""select readmissionStatus, year(periodStart) year, count(*) count
    from local.silver.encounter group by 1,2 order by 2""").toPandas(),
    x='year',
    y='count',
    color='readmissionStatus', # Differentiate lines by readmissionStatus
    title='Encounter Over the Years by ReadmissionStatus',
    labels={'year': 'Year', 'count': 'Number of Encounters'}).update_layout(
    xaxis_title='Year',
    yaxis_title='Number of Encounters',
    legend_title='Readmission Status',
    template='plotly_white'
).show()

In [ ]:
priority_encounter_df = spark.sql("""
SELECT
    e1.priority priority1,
    e2.priority priority2,
    count(*) count
FROM local.silver.encounter e1
JOIN local.silver.encounter e2
ON e1.nextEncounterId = e2.encounterID
WHERE e1.readmissionStatus == 'Readmission'
GROUP BY all
ORDER by 3 desc
""")

priority_encounter_pd = priority_encounter_df.toPandas()
all_priorities = list(set(priority_encounter_pd['priority1'].tolist() +
                          priority_encounter_pd['priority2'].tolist()))
node_map = {priority: idx for idx, priority in enumerate(all_priorities)}
priority_encounter_pd['source'] = priority_encounter_pd['priority1'].map(node_map)
priority_encounter_pd['target'] = priority_encounter_pd['priority2'].map(node_map)

table_trace = go.Table(
    header=dict(values=["Priority 1", "Priority 2", "Count"], fill_color='lightgrey', align='center'),
    cells=dict(values=[priority_encounter_pd['priority1'], priority_encounter_pd['priority2'], priority_encounter_pd['count']],
               fill_color='white', align='center')
)

sankey_trace = go.Sankey(
    node=dict(
        pad=15, # Padding between nodes
        thickness=20, # Node thickness
        line=dict(color="black", width=0.5), # Node border settings
        label=all_priorities # Node labels
    ),
    link=dict(
        source=priority_encounter_pd['source'], # Source nodes (indices)
        target=priority_encounter_pd['target'], # Target nodes (indices)
        value=priority_encounter_pd['count'] # Flow values (counts)
    )
)

fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.7, 0.3], # Adjust column widths (30% table, 70% Sankey)
    specs=[[{"type": "table"}, {"type": "sankey"}]], # Specify types for each subplot
)


fig.add_trace(sankey_trace, row=1, col=1)
fig.add_trace(table_trace, row=1, col=2)

fig.update_layout(
    title_text="Readmission Encounters: Priority Transitions Sankey Diagram and Table",
    font_size=12,
    height=500
)
fig.show()

In [ ]:
spark.sql("""
SELECT
    sourceFile,
    CASE
        WHEN duration BETWEEN 0 AND 5 THEN '0 to 5 days'
        WHEN duration BETWEEN 6 AND 10 THEN '6 to 10 days'
        WHEN duration BETWEEN 11 AND 20 THEN '11 to 20 days'
        WHEN duration BETWEEN 21 AND 30 THEN '21 to 30 days'
        WHEN duration BETWEEN 31 AND 50 THEN '31 to 50 days'
        WHEN duration BETWEEN 51 AND 100 THEN '51 to 100 days'
        WHEN duration BETWEEN 101 AND 150 THEN '101 to 150 days'
        WHEN duration BETWEEN 151 AND 200 THEN '151 to 200 days'
        WHEN duration BETWEEN 201 AND 250 THEN '201 to 250 days'
        WHEN duration BETWEEN 251 AND 300 THEN '251 to 300 days'
        ELSE 'More than 300 days'
    END AS duration_group,
    COUNT(*) AS encounter_count
FROM local.silver.encounter
GROUP BY sourceFile, duration_group
ORDER BY sourceFile, double(split_part(duration_group, ' ', 1));
""").show(50, truncate = False)

In [ ]:
px.line(
    spark.sql("""SELECT
        sourceFile,
        duration AS duration_group,
        COUNT(*) AS encounter_count
    FROM local.silver.encounter
    where duration BETWEEN 1 AND 30
    GROUP BY sourceFile, duration_group
    ORDER BY sourceFile, duration_group;""").toPandas(),
    x='duration_group', y='encounter_count', color='sourceFile',
    markers=True, title="Encounter Count by Source and Duration Group").show()

## Condition Dataset

In [ ]:
px.pie(
    spark.sql("select split_part(conditionSystem, '/', -1) conditionSystem, count(*) count from local.silver.condition group by 1").toPandas(),
    values='count', names='conditionSystem', title='Condition System Count Distribution').show()

Based on the `Condition System Count Distribution` chart, the dataset incorporates both ICD-9 and ICD-10 coding systems in condition table.

<!-- The difference between these two coding systems can introduce challenges which may affect the model performance and interpretability.

1. Inconsistent Coding:
    - ICD-10 has more granularity in specifying types of diseases, injury location, and severity.
    - ICD-9 has fewer and less specific codes than ICD-10, which means that a single ICD-9 code could map to multiple ICD-10 codes.
    - This inconsistency can create noisy features in your dataset if the same condition is coded differently depending on the coding system used. This could confuse your model, leading to reduced predictive accuracy.
2. Feature Engineering Complexity:
    - ICD-9 and ICD-10 are structured differently, both in terms of the number of codes and their specificity.
    - This complicates the creation of features related to diagnosis categories or comorbidities.
    - The features might not capture the full clinical picture, potentially leading to underfitting or overfitting
3. Data Heterogeneity:
    - Introduction of temporal bias into the model
    - This could cause the model to overestimate or underestimate readmission risks if it correlates newer coding systems with better or worse outcomes.
4. Model Interpretability:
    - Mixed coding systems make model interpretation harder, especially if using interpretable models like decision trees or logistic regression.
    - This may end up with features that are not comparable between ICD-9 and ICD-10, complicating efforts to explain your model's predictions.

> **References**
> - [Understanding the Impact of the Differences in ICD-9-CM and ICD10-CM and Its Potential Impact on Data Analysis | National Center for Health Statistics](https://www.cdc.gov/nchs/ppt/nchs2012/li-01_pickett.pdf)
> - [Overview of Coding and Classification Systems](https://www.cms.gov/cms-guide-medical-technology-companies-and-other-interested-parties/coding/overview-coding-classification-systems) -->

In [ ]:
spark.sql("""
select conditionCode, conditionDisplay, count(*) count from local.silver.condition
where conditionSystem like '%mimic-diagnosis-icd10'
group by all order by 3 desc
""").show(10, truncate = False)

In [ ]:
spark.sql("""
select conditionCode, conditionDisplay, count(*) count from local.silver.condition
where conditionSystem like '%mimic-diagnosis-icd9'
group by all order by 3 desc
""").show(10, truncate = False)

Comparing top condition for both coding systems, there are few notable common types between them.

For Example:

- ICD 9 Code 4019 `Unspecified essential hypertension` is similar to ICD 10 Code I10 `Essential (primary) hypertension`.
- ICD 9 Code V1582 `Personal history of tobacco use` is similar to ICD 10 Code Z87891 `Personal history of nicotine dependence`.
- ICD 9 Code 2724 `Other and unspecified hyperlipidemia ` is similar to ICD 10 Code E785 `Hyperlipidemia, unspecified`.

The dataset must be standardized in order to gain better accuracy of predictive model.

# 03. Data Modeling

## Condition Domain

### Standardarize Coding System

<!-- The dataset, as shown in the Condition System Count Distribution chart, includes both ICD-9 and ICD-10 coding systems in the condition table.

To standardize the dataset, one approach is to map ICD-9 codes to their ICD-10 equivalents using tools like General Equivalence Mappings (GEMs).

This conversion ensures consistency across the dataset by aligning all condition codes with the ICD-10 system, which is the current standard for coding medical diagnoses and procedures.

**ICD-10 Code Structure**
- Characters 1:3 = Indicate the category of the diagnosis
- Characters 4:6 = Indicate etiology, anatomic site, severity or other clinical detail
- Character 7 = Extension

> **References**
> - [Crosswalk or General Equivalence Mappings | NBER](https://www.nber.org/research/data/icd-9-cm-and-icd-10-cm-and-icd-10-pcs-crosswalk-or-general-equivalence-mappings) -->


In [ ]:
with open("sql_queries/silver.condition-standard.ddl.sql") as f:
    spark.sql(f.read())

In [ ]:
icd10_df = spark.sql(""" 
    SELECT 
        conditionId, encounterId, patientId, conditionCode 
    FROM local.silver.condition condition 
    WHERE condition.conditionSystem LIKE '%icd10' 
""") 
icd10_df.write.format("iceberg").mode("append").save("local.silver.condition_standard")

In [6]:
df = spark.read.csv("dataset/icd9toicd10cmgem.csv", header = True) 
df.createOrReplaceTempView("GemMapping") 

with open("sql_queries/silver.condition-standard.load.sql") as f:
    icd9_df = spark.sql(f.read())

icd9_df.write.format("iceberg").mode("append").save("local.silver.condition_standard")

In [ ]:
spark.sql(""" 
select 
    conditionID, encounterID, patientID, 
    conditionCode 
from local.silver.condition_standard 
""").show(10, truncate = False)

### Risk Categorization

<!-- **Risk categorization based on the Charlson Comorbidity Index (CCI)**

Risk categorization based on the Charlson Comorbidity Index (CCI) is a widely used method for assessing the risk of adverse outcomes in patients, particularly in healthcare settings. The CCI assigns weights to various comorbid conditions based on their severity, such as heart disease, diabetes, and cancer. These conditions are then summed to produce a total score, which can be used to predict patient outcomes, including the risk of mortality and complications. The higher the CCI score, the greater the patient’s risk. This index helps clinicians and healthcare providers identify high-risk patients, prioritize care, and make informed decisions regarding treatment plans, resource allocation, and follow-up care. It is an essential tool for improving patient outcomes and optimizing healthcare management.

| Condition                             | Weight | Risk Level |
|---------------------------------------|--------|------------|
| Peripheral vascular disease           | 1      | Low        |
| Cerebrovascular disease               | 1      | Low        |
| Chronic pulmonary disease             | 1      | Low        |
| Congestive heart failure              | 1      | Low        |
| Rheumatic disease                     | 1      | Low        |
| Diabetes without chronic complication | 1      | Low        |
| Mild liver disease                    | 1      | Low        |
| Peptic ulcer disease                  | 1      | Low        |
| Renal disease                         | 1      | Low        |
| Myocardial infarction                 | 2      | Moderate   |
| Hemiplegia or paraplegia              | 2      | Moderate   |
| Malignancy                            | 2      | Moderate   |
| Diabetes with chronic complication    | 2      | Moderate   |
| Moderate or severe liver disease      | 3      | High       |
| Metastatic solid tumour               | 6      | High       |
| AIDS/HIV                              | 6      | High       |

> **References**
> - [Charlson Comorbidity Index Calculator](https://www.mdcalc.com/calc/3917/charlson-comorbidity-index-cci)
> - [Calculation of the Charlson Comorbidity Index score](https://www.ncbi.nlm.nih.gov/books/NBK587905/)
> - [ICD-10-CM to CMS-HCC Crosswalk](https://provider.amerigroup.com/dam/publicdocuments/ALL_CARE_CMSHCCRAModel_mrdcoding_tips.pdf)
> - [CCI ICD-10 codes Mapping](https://www.psychiatryinvestigation.org/upload/media/pi-2021-0223-suppl1.pdf) -->

In [ ]:
with open("sql_queries/silver.condition-risk.ddl.sql") as f:
    spark.sql(f.read())

In [4]:
with open("sql_queries/silver.condition-risk.load.sql") as f:
    condition_risk_df = spark.sql(f.read())

condition_risk_df.write.format("iceberg").mode("overwrite").save("local.silver.condition_risk")

In [ ]:
spark.sql("select charlsonCategory, riskLevel, count(*) from local.silver.condition_risk group by 1,2").show(20, truncate = False)

### Case-Based Aggregation

In [ ]:
with open("sql_queries/silver.condition-rollup.ddl.sql") as f:
    spark.sql(f.read())

In [30]:
with open("sql_queries/silver.condition-rollup.load.sql") as f:
    condition_rollup_df = spark.sql(f.read())

condition_rollup_df.show(10, truncate = False)
condition_rollup_df.write.format("iceberg").mode("overwrite").save("local.silver.condition_rollup")

## Consolidated Encounter

<!-- The Gold Layer represents the refined stage in the Medallion Architecture, where data from the Silver Layer is consolidated and fully processed for advanced analytics and reporting. The Gold Table serves as the foundation for high-level analysis, enabling analysts, and researchers to derive meaningful insights that drive operational efficiency, improve patient outcomes, and support value-based care initiatives.

The Encounter Gold Table integrates and structures the most critical data points related to patient encounters, ensuring that it is clean, accurate, and ready for decision-making. The table has various data points (such as patient demographics, encounter details, clinical diagnoses) merged and aggregated into a comprehensive, easily accessible format.  -->

In [11]:
with open("sql_queries/gold.encounter.ddl.sql") as f:
    spark.sql(f.read())

In [12]:
with open("sql_queries/gold.encounter.load.sql") as f:
    gold_encounter_df = spark.sql(f.read())

gold_encounter_df.write.format("iceberg").mode("overwrite").save("local.gold.encounter")

# 04. Data Profiling

In [139]:
parquet_location = 'dataset/encounter.parquet'
# gold_encounter_df = spark.read.format("iceberg").load("local.gold.encounter").toPandas()
# gold_encounter_df.to_parquet(parquet_location)
gold_encounter_df = pd.read_parquet(parquet_location)

# Convert columns to the category data type. 
cols_to_convert = ["gender", "ageGroup", "maritalStatus", "encounterStatus", "encounterClass", "admitSource", "encounterType", "priority", "readmissionStatus"]
for col in cols_to_convert:
    gold_encounter_df[col] = gold_encounter_df[col].astype('category')

non_features_columns = ['encounterId','patientId','encounterTs','encounterStatus', 'encounterType']
gold_encounter_df = gold_encounter_df.drop(non_features_columns, axis=1)

## DataFrame Summary

In [ ]:
gold_encounter_df.info(verbose=False)
gold_encounter_df.describe()
s_datatype = pd.Series(gold_encounter_df.dtypes, name='DataType')
s_uniquecount = pd.Series(gold_encounter_df.nunique(), name='Unique Count')
s_count = pd.Series(gold_encounter_df.count(), name='Total Count')
s_nullcount = pd.Series(gold_encounter_df.isnull().sum(), name='Null Values')
report_df = pd.concat([s_datatype, s_uniquecount, s_count, s_nullcount], axis=1)
display(report_df)

## Imbalance Ratio

- An imbalance ratio of 1.54 is moderate and may still impact model performance.
- Possible Cause: Readmissions (the minority class) are typically less frequent than non-readmissions (the majority class)
- Impact of Class Imbalance: Bias toward Majority Class (Non-readmissions)

Measures to solve imbalance ratio:
- Undersampling the Majority Class: Remove some non-readmitted patients to balance the dataset. However, this may lead to the loss of useful information.
- Oversampling the Minority Class: Increase the readmitted patients to balance the dataset. Help the model learn better about the minority class and prevent it from being biased toward the majority class.

In [ ]:
target_distribution = gold_encounter_df.groupby('readmissionStatus').size()
majority_class_count = target_distribution.max()
minority_class_count = target_distribution.min()
imbalance_ratio = majority_class_count/minority_class_count
print("Imbalance Ratio: {:.2f}".format(imbalance_ratio))

target_distribution.plot(kind='bar')
plt.title('Class Distribution')
plt.ylabel('Frequency')
plt.xlabel('Class')
plt.show()

# 05. Feature Exploration

### Readmission Status by Categorical Variables

In [ ]:
# Group by categorical variable and count occurrences
grouped_data = {}
for column in ['gender', 'ageGroup', 'maritalStatus', 'encounterClass', 'admitSource', 'priority']:
    grouped_data[column] = gold_encounter_df.groupby([column, 'readmissionStatus']).size().reset_index(name='counts')

fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(18, 10))
fig.suptitle('Count of Readmission Status by Categorical Variables', fontsize=16)
for i, column in enumerate(grouped_data.keys()):
    row = i // 3  # Calculate row index
    col = i % 3   # Calculate column index
    ax = axes[row, col]
    for category in grouped_data[column][column].unique():
        category_data = grouped_data[column][grouped_data[column][column] == category]
        ax.bar(category_data['readmissionStatus'], category_data['counts'], label=category)
    ax.set_xlabel('Readmission Status')
    ax.set_ylabel('Count')
    ax.set_title(f'{column}')
    ax.legend(title=column)

plt.tight_layout()
plt.show()

## Bar Chart Age Group Size

In [ ]:
df = gold_encounter_df
order = ['young adults' ,'young adulthood' ,'early-middle age' ,'late-middle age' ,'mid-old age' ,'senior-old age' ,'very senior-old' ,'centenarians' ] 

df['ageGroup'] = pd.Categorical(df['ageGroup'], categories=order, ordered=True)
grouped_df = df.groupby('ageGroup').size().reset_index(name='count')

plt.figure(figsize=(18, 6))
plt.bar(grouped_df['ageGroup'], grouped_df['count'])
plt.xlabel('Age Group')
plt.ylabel('Count')
plt.title('Count of Individuals by Age Group')
plt.show()

## Heatmap Risk Scores by Age Group And Rick Score Tens

In [ ]:
df = gold_encounter_df[gold_encounter_df['riskScore']> 0]
df['scoreTens'] = df['riskScore'] // 10 * 10

grouped_df = df.groupby(['ageGroup', 'scoreTens']).size().reset_index(name='count')

pivot_df = grouped_df.pivot_table(index='ageGroup', columns='scoreTens', values='count', fill_value=0)

plt.figure(figsize=(10, 8))
plt.imshow(pivot_df, cmap='viridis', interpolation='nearest')
plt.xticks(range(len(pivot_df.columns)), pivot_df.columns)
plt.yticks(range(len(pivot_df.index)), pivot_df.index)
plt.xlabel('Risk Score Tens')
plt.ylabel('Age Group')
plt.title('Count of Risk Scores by Age Group and Risk Score Tens')
plt.colorbar()

for i in range(len(pivot_df.index)):
    for j in range(len(pivot_df.columns)):
        plt.text(j, i, str(pivot_df.values[i, j]), ha='center', va='center', color='w')

plt.show()

## Heatmap High/Low/Moderate Risk by Age Group

In [ ]:
conditions = ['highRiskCondition', 'moderateRiskCondition', 'lowRiskCondition']
titles = ['High Risk', 'Moderate Risk', 'Low Risk']

fig, axes = plt.subplots(3, 1, figsize=(12, 12))

for i, condition in enumerate(conditions):
    filtered_df = df[df[condition] > 0]
    filtered_df['scoreTens'] = filtered_df[condition]
    grouped_df = filtered_df.groupby(['ageGroup', 'scoreTens']).size().reset_index(name='count')
    pivot_df = grouped_df.pivot_table(index='ageGroup', columns='scoreTens', values='count', fill_value=0)

    axes[i].imshow(pivot_df, cmap='viridis', interpolation='nearest')
    axes[i].set_xticks(range(len(pivot_df.columns)), pivot_df.columns)
    axes[i].set_yticks(range(len(pivot_df.index)), pivot_df.index)
    axes[i].set_title(f'Count of {titles[i]} Scores by Age Group')
plt.tight_layout()
plt.show()

## Heatmap High Risk by Age Group (Detail View)

In [ ]:
df = gold_encounter_df[gold_encounter_df['highRiskCondition'] > 0]
df['scoreTens'] = df['highRiskCondition']

grouped_df = df.groupby(['ageGroup', 'scoreTens']).size().reset_index(name='count')

pivot_df = grouped_df.pivot_table(index='ageGroup', columns='scoreTens', values='count', fill_value=0)

plt.figure(figsize=(12, 8))
plt.imshow(pivot_df, cmap='viridis', interpolation='nearest')
plt.xticks(range(len(pivot_df.columns)), pivot_df.columns)
plt.yticks(range(len(pivot_df.index)), pivot_df.index)
plt.xlabel('High Risk Count')
plt.ylabel('Age Group')
plt.title('Count of High Risk by Age Group (Detail View)')
plt.colorbar()

for i in range(len(pivot_df.index)):
    for j in range(len(pivot_df.columns)):
        plt.text(j, i, str(pivot_df.values[i, j]), ha='center', va='center', color='w')

plt.show()

## Heatmap Length Stay by Age Group

In [ ]:
df = gold_encounter_df[(gold_encounter_df['stayLength']> 0) & (gold_encounter_df['stayLength'] <= 60)]
df['scoreTens'] = df['stayLength'] // 5 * 5

grouped_df = df.groupby(['ageGroup', 'scoreTens']).size().reset_index(name='count')

pivot_df = grouped_df.pivot_table(index='ageGroup', columns='scoreTens', values='count', fill_value=0)

plt.figure(figsize=(20, 8))
plt.imshow(pivot_df, cmap='viridis', interpolation='nearest')
plt.xticks(range(len(pivot_df.columns)), pivot_df.columns)
plt.yticks(range(len(pivot_df.index)), pivot_df.index)
plt.xlabel('Risk Score Tens')
plt.ylabel('Age Group')
plt.title('Count of Length Stay by Age Group')
plt.colorbar()

for i in range(len(pivot_df.index)):
    for j in range(len(pivot_df.columns)):
        plt.text(j, i, str(pivot_df.values[i, j]), ha='center', va='center', color='w')

plt.show()

# 06. Data Preprocessing

In [140]:
# pip install imblearn
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [ ]:
gold_encounter_df['readmissionStatus'] = gold_encounter_df['readmissionStatus'].map({'Readmission': 1, 'No Readmission': 0})
gold_encounter_df

## One-Hot Encoding

- Encoding categorical variables converts categorical variables (text labels) into numerical values (integer labels).
- Many machine learning algorithms expect numerical input. Categorical variables, which represent groups or categories (e.g., "red," "blue," "green"), cannot be directly used by these algorithms.
- **How it works**: For each category within a feature, a new binary column is created. A "1" is placed in the corresponding column for a specific data point if that category applies, and "0" otherwise.

Why One-Hot Encoding over Label Encoding:
- Avoids Implicit Ordinal Relationships (No Implicit Ranking)
- Interpretability

In [ ]:
categorical_columns = ['gender','ageGroup','maritalStatus','encounterClass','admitSource','priority']
encoder = OneHotEncoder(sparse_output=False)

# Apply one-hot encoding to the categorical columns
one_hot_encoded = encoder.fit_transform(gold_encounter_df[categorical_columns])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

target_df = pd.concat([gold_encounter_df, one_hot_df], axis=1)
target_df = target_df.drop(categorical_columns, axis=1)
target_df

## Oversampling the Minority Class

In [ ]:
X = target_df.drop(columns=['readmissionStatus'])
y = target_df['readmissionStatus']

print("Class distribution before oversampling:")
print(Counter(y))

# Apply SMOTE to oversample the minority class
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Check the class distribution after oversampling
print("Class distribution after oversampling:")
print(Counter(y_resampled))

## Scaling Numerical Features

- Ensures all numerical features in a dataset are on a similar scale.
- Many machine learning algorithms are sensitive to the scale of the input features, particularly distance-based algorithms like k-Nearest Neighbors (k-NN) and gradient descent-based algorithms.
- Features with larger scales can dominate the learning process, leading to biased models. Scaling ensures that all features contribute equally to the model's learning.

In [ ]:
cols_to_scale = ['stayLength', 'riskScore', 'highRiskCondition', 'moderateRiskCondition', 'lowRiskCondition'] 

scaler = StandardScaler()
X_resampled[cols_to_scale] = scaler.fit_transform(X_resampled[cols_to_scale])

X_resampled

## Pearson Correlation Coefficient

> Moderate positive correlation between `risk score` and `risk conditions`

In [ ]:
df = X_resampled[["stayLength","riskScore","highRiskCondition","moderateRiskCondition","lowRiskCondition"]]
correlation_matrix = df.corr(method='pearson')

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", cbar=True)
plt.title('Correlation Matrix Heatmap')
plt.show()

# 07. Model Selection

In [152]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

In [147]:
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.275, random_state=123
)